# 金融异常检测任务

## 1. 实验介绍

反欺诈是金融行业永恒的主题，在互联网金融信贷业务中，数字金融反欺诈技术已经得到广泛应用并取得良好效果，这其中包括了近几年迅速发展并在各个领域
得到越来越广泛应用的神经网络。本项目以互联网智能风控为背景，从用户相互关联和影响的视角，探索满足风控反欺诈领域需求的，可拓展、高效的神经
网络应用方案，从而帮助更好地识别欺诈用户。

本项目主要关于实现预测模型(**项目用图神经网络举例，具体实现可以使用其他模型**)，进行节点异常检测任务，并验证模型精度。而本项目基于的数据集[DGraph](https://dgraph.xinye.com/introduction)，[DGraph](https://dgraph.xinye.com/introduction)
是大规模动态图数据集的集合，由真实金融场景中随着时间演变事件和标签构成。

### 1.1 实验目的

- 了解如何使用Pytorch进行神经网络训练
- 了解如何使用Pytorch-geometric等图网络深度学习库进行简单图神经网络设计(推荐使用GAT, GraphSAGE模型)。
- 了解如何利用MO平台进行模型性能评估。

### 1.2 预备知识
- 具备一定的深度学习理论知识，如卷积神经网络、损失函数、优化器，训练策略等。
- 了解并熟悉Pytorch计算框架。
- 学习Pytorch-geometric，请前往：https://pytorch-geometric.readthedocs.io/en/latest/
    
### 1.3实验环境
- numpy = 1.26.4  
- pytorch = 2.3.1  
- torch_geometric = 2.5.3  
- torch_scatter = 2.1.2  
- torch_sparse = 0.6.18  

## 2. 实验内容

### 2.1 数据集信息
DGraph-Fin 是一个由数百万个节点和边组成的有向无边权的动态图。它代表了Finvolution Group用户之间的社交网络，其中一个节点对应一个Finvolution 用户，从一个用户到另一个用户的边表示**该用户将另一个用户视为紧急联系人**。
下面是`位于dataset/DGraphFin目录`的DGraphFin数据集的描述:
```
x:  20维节点特征向量
y:  节点对应标签，一共包含四类。其中类1代表欺诈用户而类0代表正常用户(实验中需要进行预测的两类标签)，类2和类3则是背景用户，即无需预测其标签。
edge_index:  图数据边集,每条边的形式(id_a,id_b)，其中ids是x中的索引
edge_type: 共11种类型的边
edge_timestamp: 脱敏后的时间戳
train_mask, valid_mask, test_mask: 训练集，验证集和测试集掩码
```
本预测任务为识别欺诈用户的节点预测任务,只需要将欺诈用户（Class 1）从正常用户（Class 0）中区分出来。需要注意的是，其中测试集中样本对应的label**均被标记为-100**。

### 2.2 导入相关包

导入相应模块，设置数据集路径、设备等。

### 2.3 数据处理

在使用数据集训练网络前，首先需要对数据进行归一化等预处理，如下：

这里我们可以查看数据各部分维度

In [1]:
from typing import Union

from torch import Tensor
from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class SAGE(torch.nn.Module):
    def __init__(self
                 , in_channels
                 , hidden_channels
                 , out_channels
                 , num_layers
                 , dropout
                 , batchnorm=True):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.batchnorm = batchnorm
        if self.batchnorm:
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            if self.batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        if self.batchnorm:
            for bn in self.bns:
                bn.reset_parameters()

    def forward(self, x, edge_index: Union[Tensor, SparseTensor]):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            if self.batchnorm:
                x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)


In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from utils import DGraphFin
import torch_geometric.transforms as T
from utils.evaluator import Evaluator

path = './datasets/632d74d4e2843a53167ee9a1-momodel/'
save_dir = './results/'  # 模型保存路径
dataset_name = 'DGraph'
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor())

nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2  # 本实验中仅需预测类0和类1
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()  # 将有向图转化为无向图


if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  # 划分训练集，验证集

In [3]:
eval_metric = 'auc'
evaluator = Evaluator(eval_metric)

In [4]:
def train(model, data, train_idx, optimizer):
    # data.y is labels of shape (N, )
    model.train()

    optimizer.zero_grad()

    out = model(data.x, data.adj_t)[train_idx]

    loss = F.nll_loss(out, data.y[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [5]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # data.y is labels of shape (N, )
    model.eval()

    out = model(data.x, data.adj_t)

    y_pred = out.exp()  # (N,num_classes)

    losses, eval_results = dict(), dict()
    for key in ['train', 'valid']:
        node_id = split_idx[key]
        losses[key] = F.nll_loss(out[node_id], data.y[node_id]).item()
        eval_results[key] = evaluator.eval(data.y[node_id], y_pred[node_id])[eval_metric]

    return eval_results, losses, y_pred

In [1]:
import argparse
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from utils import DGraphFin
from utils.utils import prepare_folder
import numpy as np

args = argparse.Namespace(
    log_steps=10,
    epochs=200,
    runs=10,
)

device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

ModuleNotFoundError: No module named 'logger'

In [7]:
model = SAGE(
    in_channels=data.x.size(-1),
    out_channels=nlabels,
    **{
        'num_layers': 2,
        'hidden_channels': 128,
        'dropout': 0,
        'batchnorm': False
    }).to(device)

In [8]:
import gc
gc.collect()
print(sum(p.numel() for p in model.parameters()))

5762


In [9]:
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-7)
best_valid = 0
min_valid_loss = 1e8
best_out = None

In [ ]:
for epoch in range(1, 300):
    loss = train(model, data, split_idx['train'], optimizer)
    eval_results, losses, out = test(model, data, split_idx, evaluator)
    train_eval, valid_eval = eval_results['train'], eval_results['valid']
    train_loss, valid_loss = losses['train'], losses['valid']

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_out = out.cpu()
        torch.save(model.state_dict(), save_dir+'/model.pt')

    if epoch % args.log_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_eval:.3f}%, '
              f'Valid: {100 * valid_eval:.3f}% ')

with torch.no_grad():
    model.eval()
    res = model(data.x, data.adj_t)
    res = res.exp()
    
res.cpu().numpy().tofile('res.pkl')

/home/jovyan/.virtualenvs/basenv/lib/python3.9/site-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())


### 2.6 模型预测

## 3. 作业评分

**作业要求**：    
                         
1. 请加载你认为训练最佳的模型（不限于图神经网络)
2. 提交的作业包括【程序报告.pdf】和代码文件。

**注意：**
          
1. 在训练模型等过程中如果需要**保存数据、模型**等请写到 **results** 文件夹，如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请务必将模型保存在 **results** 文件夹下。
2. 训练出自己最好的模型后，先按照下列 cell 操作方式实现 NoteBook 加载模型测试；请测试通过在进行【系统测试】。
3. 点击左侧栏`提交作业`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
4. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
5. 请加载你认为训练最佳的模型，即请按要求填写**模型路径**。
6. `predict()`函数的输入和输出请不要改动。

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [14]:
## 生成 main.py 时请勾选此 cell
from utils import DGraphFin
from utils.evaluator import Evaluator
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.data import Data
import numpy as np
import os

# path = './datasets/632d74d4e2843a53167ee9a1-momodel/'
# save_dir = './results/'  # 模型保存路径
# dataset_name = 'DGraph'
# dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor())

# nlabels = dataset.num_classes
# if dataset_name in ['DGraph']:
#     nlabels = 2  # 本实验中仅需预测类0和类1
# data = dataset[0]
# data.adj_t = data.adj_t.to_symmetric() 

# # 确定设备：如果有 GPU 则使用 GPU，否则使用 CPU
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# # 加载数据并将其移动到设备上
# data = data.to(device)

# # 定义模型
# model = SAGE(
#     in_channels=data.x.size(-1),
#     out_channels=nlabels,
#     **{
#         'num_layers': 2,
#         'hidden_channels': 128,
#         'dropout': 0,
#         'batchnorm': False
#     }).to(device)

# # 加载模型权重时，处理 CPU 和 GPU 的情况
# if device.type == 'cpu':
#     # 如果当前设备是 CPU，则将模型映射到 CPU
#     model.load_state_dict(torch.load('./results/model.pt', map_location=torch.device('cpu')))
# else:
#     # 如果当前设备是 GPU，则直接加载到 GPU
#     model.load_state_dict(torch.load('./results/model.pt'))
    
res = np.fromfile('./results/res.pkl', dtype=np.float32).reshape(-1, 2)



def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """

    # 模型预测时，测试数据已经进行了归一化处理


    return res[node_id]
